In [1]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

/tmp/ipykernel_2464980/3212585940.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
df = pd.read_excel('fraud_email_尼日利亚骗局标注.xlsx', sheet_name=1)

# 数据清洗
df = df[:49]
df_names = df.columns.to_list()[3:]
for df_name in df_names:
    df[df_name] = df[df_name] == 1.0
df.head()

,英文内容,钓鱼邮件,中文翻译,权威偏误,光环效应,双曲贴合,好奇心效应,紧迫性偏差,零风险偏误
0,Supply Quality China's EXCLUSIVE dimensions at...,1,供应质量中国的独家尺寸，价格无敌。尊敬的先生，我们很高兴地通知您，作为中国最大的出口和进口石...,False,True,False,True,False,False
1,"Dear Friend,Greetings to you.I wish to accost ...",1,亲爱的朋友，\n\n向您问好。\n\n我希望向您提出一个请求，这将对我们双方都有巨大的好处。...,True,True,False,True,True,False
2,MR. CHEUNG PUIHANG SENG BANK LTD.DES VOEUX RD....,1,尊敬的先生/女士，\n\n让我先自我介绍。我是Cheung Pui，恒生银行有限公司运营总监...,False,True,False,True,False,False
3,Greetings from barrister Robert Williams=2CDea...,1,来自罗伯特·威廉姆斯律师的问候，\n\n亲爱的朋友，我知道我的信件会让你感到惊讶，因为我们之...,True,True,False,True,False,True
4,SOLICITING FOR A BUSINESS VENTURE AND PARTNESH...,1,寻求商业合作与伙伴关系\n\n尊敬的先生，\n\n我是已故扎伊尔共和国总统蒙博托·塞塞·塞科...,True,False,False,True,False,False


In [3]:
# 构建数据集
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(df['英文内容'])

# 定义模型
classifier = DecisionTreeClassifier()

# 定义并应用五折交叉验证
kf = StratifiedKFold(n_splits=5)  # 根据需要调整折数

for df_name in df_names:
    y = df[df_name]
    y_pred = cross_val_predict(classifier, X, y, cv=kf)
    
    # 计算混淆矩阵
    cm = confusion_matrix(y, y_pred)# 构建数据集
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(df['英文内容'])

# 定义模型
classifier = DecisionTreeClassifier()

# 定义并应用五折交叉验证
kf = StratifiedKFold(n_splits=5)  # 根据需要调整折数

for df_name in df_names:
    y = df[df_name]
    y_pred = cross_val_predict(classifier, X, y, cv=kf)
    
    # 计算混淆矩阵
    cm = confusion_matrix(y, y_pred)
    
    # 检查混淆矩阵的形状，并适当处理
    if cm.shape == (2, 2):
        TN, FP, FN, TP = cm.ravel()
    else:
        # 如果混淆矩阵不是2x2，需要根据你的数据调整下面的代码
        TN, FP, FN, TP = 0, 0, 0, 0
        if len(cm) == 1:  # 假设只有一个类别被预测
            if y_pred[0] == y.iloc[0]:  # 如果预测的类别与实际类别相同
                TP = cm[0][0]  # 所有预测都是真正类
            else:
                TN = cm[0][0]  # 所有预测都是真阴类
    
    # 计算指标
    TPR = TP / (TP + FN) if (TP + FN) else 0  # 真正率或召回率
    FPR = FP / (FP + TN) if (FP + TN) else 0  # 假阳性率
    precision = precision_score(y, y_pred, zero_division=0)
    recall = recall_score(y, y_pred, zero_division=0)
    f1 = f1_score(y, y_pred, zero_division=0)
    
    # 打印指标
    print(f'交叉验证:{df_name}')
    print(f'真正率(TPR)/召回率: {TPR:.2%}')
    print(f'假阳性率(FPR): {FPR:.2%}')
    print(f'精确度: {precision:.2%}')
    print(f'召回率: {recall:.2%}')
    print(f'F1分数: {f1:.2%}\n')
    
    # 检查混淆矩阵的形状，并适当处理
    if cm.shape == (2, 2):
        TN, FP, FN, TP = cm.ravel()
    else:
        # 根据实际的类别分布调整，这里是一个简化的处理方式
        TN, FP, FN, TP = 0, 0, 0, 0
        if len(cm) == 1:  # 假设只有一个类别被预测
            if y_pred[0] == y.iloc[0]:  # 假设预测的类别与实际类别相同
                TP = cm[0][0]  # 所有预测都是真正类
            else:
                TN = cm[0][0]  # 所有预测都是真阴类

/data/yyl/conda_envs/huggingface/lib/python3.9/site-packages/sklearn/metrics/_classification.py:386: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
/data/yyl/conda_envs/huggingface/lib/python3.9/site-packages/sklearn/metrics/_classification.py:386: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
/data/yyl/conda_envs/huggingface/lib/python3.9/site-packages/sklearn/metrics/_classification.py:386: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


交叉验证:权威偏误
真正率(TPR)/召回率: 58.33%
假阳性率(FPR): 44.00%
精确度: 56.00%
召回率: 58.33%
F1分数: 57.14%

交叉验证:光环效应
真正率(TPR)/召回率: 73.08%
假阳性率(FPR): 8.70%
精确度: 90.48%
召回率: 73.08%
F1分数: 80.85%

交叉验证:双曲贴合
真正率(TPR)/召回率: 100.00%
假阳性率(FPR): 0.00%
精确度: 0.00%
召回率: 0.00%
F1分数: 0.00%

交叉验证:好奇心效应
真正率(TPR)/召回率: 100.00%
假阳性率(FPR): 0.00%
精确度: 100.00%
召回率: 100.00%
F1分数: 100.00%

交叉验证:紧迫性偏差
真正率(TPR)/召回率: 56.25%
假阳性率(FPR): 18.18%
精确度: 60.00%
召回率: 56.25%
F1分数: 58.06%

交叉验证:零风险偏误
真正率(TPR)/召回率: 52.94%
假阳性率(FPR): 34.38%
精确度: 45.00%
召回率: 52.94%
F1分数: 48.65%



/data/yyl/conda_envs/huggingface/lib/python3.9/site-packages/sklearn/metrics/_classification.py:386: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


In [18]:
# 垃圾邮件
df = pd.read_excel('Phishing_Email_标注.xlsx', sheet_name=1)

# 数据清洗
df = df[:90]
df_names = df.columns.to_list()[4:]
for df_name in df_names:
    df[df_name] = df[df_name] == 1.0
df.head()

,Unnamed: 0,Email Text,Email Type,中文翻译,权威偏误,光环效应,双曲贴合,好奇心效应,紧迫性偏差,零风险偏误,稀缺性偏见
0,3,\nHello I am your hot lil horny toy.\n I am...,Phishing Email,你好，我是你的性感玩具。\n 我是你梦寐以求的人、\n 我是一个思想非常开放的人...,False,False,False,True,False,False,False
1,7,"entourage , stockmogul newsletter ralph velez ...",Phishing Email,"entourage , stockmogul newsletter ralph velez ...",False,True,False,True,False,False,False
2,8,"we owe you lots of money dear applicant , afte...",Phishing Email,我们欠您很多钱，亲爱的申请人，在收到您的申请后，经过进一步审查，您目前的抵押贷款有资格获得3...,False,False,False,True,False,False,False
3,10,make her beg you to give it to her everynight ...,Phishing Email,如果您不相信我，请相信《纽约时报》、《新闻周刊》、《时代周刊》以及全网许多人的评价。3 + ...,False,True,False,True,False,True,False
4,22,\nQuestion?Do you want a different job?\nDo yo...,Phishing Email,问题：你想换一份工作吗？\n您想自己当老板吗？\n你需要额外收入吗？\n您需要开始新的生活吗...,False,True,True,True,False,False,False


In [32]:
from sklearn.model_selection import cross_val_score

# 构建数据集
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(df['Email Text'])

# 定义模型
classifier = DecisionTreeClassifier()

# 定义并应用五折交叉验证
kf = StratifiedKFold(n_splits=5)

for df_name in df_names:
    y = df[df_name]
    scores = cross_val_score(classifier, X, y, cv=kf)
    print('交叉验证:{} \n交叉验证得分:{} \n交叉验证平均得分:{}\n'.format(df_name, scores, scores.mean()))

交叉验证:权威偏误 
交叉验证得分:[0.83333333 0.88888889 0.88888889 0.88888889 0.88888889] 
交叉验证平均得分:0.8777777777777779

交叉验证:光环效应 
交叉验证得分:[0.88888889 0.66666667 0.72222222 0.77777778 0.77777778] 
交叉验证平均得分:0.7666666666666666

交叉验证:双曲贴合 
交叉验证得分:[0.66666667 0.88888889 0.88888889 0.77777778 0.72222222] 
交叉验证平均得分:0.7888888888888889

交叉验证:好奇心效应 
交叉验证得分:[0.55555556 0.5        0.5        0.83333333 0.77777778] 
交叉验证平均得分:0.6333333333333333

交叉验证:紧迫性偏差 
交叉验证得分:[0.83333333 0.94444444 0.72222222 0.88888889 0.88888889] 
交叉验证平均得分:0.8555555555555555

交叉验证:零风险偏误 
交叉验证得分:[0.94444444 0.88888889 0.88888889 0.83333333 0.83333333] 
交叉验证平均得分:0.8777777777777779

交叉验证:稀缺性偏见 
交叉验证得分:[0.55555556 0.5        0.72222222 0.55555556 0.83333333] 
交叉验证平均得分:0.6333333333333333



In [ ]:
# 构建数据集
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(df['Email Text'])

# 定义模型
classifier = DecisionTreeClassifier()

# 定义并应用五折交叉验证
kf = StratifiedKFold(n_splits=5)  # 根据需要调整折数

for df_name in df_names:
    y = df[df_name]
    y_pred = cross_val_predict(classifier, X, y, cv=kf)
    
    # 计算混淆矩阵
    cm = confusion_matrix(y, y_pred)# 构建数据集
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(df['Email Text'])

# 定义模型
classifier = DecisionTreeClassifier()

# 定义并应用五折交叉验证
kf = StratifiedKFold(n_splits=10)  # 根据需要调整折数

for df_name in df_names:
    y = df[df_name]
    y_pred = cross_val_predict(classifier, X, y, cv=kf)
    
    # 计算混淆矩阵
    cm = confusion_matrix(y, y_pred)
    
    # 检查混淆矩阵的形状，并适当处理
    if cm.shape == (2, 2):
        TN, FP, FN, TP = cm.ravel()
    else:
        # 如果混淆矩阵不是2x2，需要根据你的数据调整下面的代码
        TN, FP, FN, TP = 0, 0, 0, 0
        if len(cm) == 1:  # 假设只有一个类别被预测
            if y_pred[0] == y.iloc[0]:  # 如果预测的类别与实际类别相同
                TP = cm[0][0]  # 所有预测都是真正类
            else:
                TN = cm[0][0]  # 所有预测都是真阴类
    
    # 计算指标
    TPR = TP / (TP + FN) if (TP + FN) else 0  # 真正率或召回率
    FPR = FP / (FP + TN) if (FP + TN) else 0  # 假阳性率
    precision = precision_score(y, y_pred, zero_division=0)
    recall = recall_score(y, y_pred, zero_division=0)
    f1 = f1_score(y, y_pred, zero_division=0)
    
    # 打印指标
    print(f'交叉验证:{df_name}')
    print(f'真正率(TPR)/召回率: {TPR:.2%}')
    print(f'假阳性率(FPR): {FPR:.2%}')
    print(f'精确度: {precision:.2%}')
    print(f'召回率: {recall:.2%}')
    print(f'F1分数: {f1:.2%}\n')
    
    # 检查混淆矩阵的形状，并适当处理
    if cm.shape == (2, 2):
        TN, FP, FN, TP = cm.ravel()
    else:
        # 根据实际的类别分布调整，这里是一个简化的处理方式
        TN, FP, FN, TP = 0, 0, 0, 0
        if len(cm) == 1:  # 假设只有一个类别被预测
            if y_pred[0] == y.iloc[0]:  # 假设预测的类别与实际类别相同
                TP = cm[0][0]  # 所有预测都是真正类
            else:
                TN = cm[0][0]  # 所有预测都是真阴类